In [1]:
import tensorflow as tf
import numpy as np
import sys
import os
from six.moves import cPickle as pickle
import functools
#OPTIONAL FOR DEBUGGING
#import import_ipynb

In [2]:
#Parameters of photo
img_size = 256
num_channels = 4
num_classes = 2
batch_size = 100
#Hyper parameters for network
learning_rate = 1e-4
params = {"learning_rate": learning_rate,
          "img_size": img_size,
          "num_channels":num_channels,
          "num_classes":num_classes,
          "batch_size":batch_size}

#### Following code serves for nice and smart defyning model as reusable code

In [19]:
def doublewrap(function):
    """
    A decorator decorator, allowing to use the decorator to be used without
    parentheses if not arguments are provided. All arguments must be optional.
    """
    @functools.wraps(function)
    def decorator(*args, **kwargs):
        if len(args) == 1 and len(kwargs) == 0 and callable(args[0]):
            return function(args[0])
        else:
            return lambda wrapee: function(wrapee, *args, **kwargs)
    return decorator

@doublewrap
def define_scope(function, scope = None, *args, **kwargs):
    
    attribute = '_cache_' + function.__name__
    name = scope or function.__name__
    
    @property
    @functools.wraps(function)
    def decorator(self):
        if not hasattr(self, attribute):
            with tf.variable_scope(name, *args, **kwargs):
                setattr(self, attribute, function(self))
        return getattr(self, attribute)
    
    return decorator

##### Data loading Block, change that block from .ipynb to .py

In [3]:
from my_load_data import load_dataset_fn
x,y = load_dataset_fn('train')
print(x.shape, y.shape)


(80, 256, 256, 3) (80, 2)


##### Defining model as reusable function

In [12]:
class Model:
    
    def __init__(self, data, labels, params, mode=None):
        self.data = data
        self.target = labels
        self.img_size = params["img_size"]
        self.num_channels = params["num_channels"]
        self.num_classes = params["num_classes"]
        self.mode = mode
        self.prediction
        self.optimize
        self.error
        
    @define_scope
    def prediction(self):
        #INPUT LAYER
        input_layer = tf.reshape(self.data,[-1, self.img_size, self.img_size, self.num_channels])
        #1 conv layer
        conv1 = tf.layers.conv2d(inputs = input_layer, 
                                 filters = 16,
                                 kernel_size = 5,
                                 strides = 1,
                                 padding = "same",
                                 activation = tf.nn.relu)
        #1 pool layer, img size reduced by 1/4
        pool1 = tf.layers.max_pooling2d(inputs=conv1,
                                        pool_size = 2, 
                                        strides = 2,
                                        padding = "same")


        pool_1_flat = tf.reshape(pool1,[-1,int(self.img_size * self.img_size * 16 / 4)])


        dense = tf.layers.dense(inputs = pool_1_flat,
                                units = 128,
                                activation = tf.nn.relu)

        dense2 = tf.layers.dense(inputs = dense,
                                 units = 64,
                                 activation = tf.nn.relu)

        logits = tf.layers.dense(inputs = dense2,
                                 units = self.num_classes,
                                 activation = tf.nn.relu)




        # Softmax output of the neural network.
        #[10,x] tesnor
        y_pred = tf.nn.softmax(logits=logits)
        y_pred_cls = tf.argmax(y_pred, axis=1)
        return y_pred
    
    
    
    @define_scope
    def optimize(self):
        cross_entropy =  tf.nn.softmax_cross_entropy_with_logits_v2(labels=self.target, logits=self.prediction)
        optimizer = tf.train.AdamOptimizer(learning_rate = params["learning_rate"])
        return optimizer.minimize(cross_entropy)

    @define_scope
    def error(self):
        mistakes = tf.not_equal(
            tf.argmax(self.target, 1), tf.argmax(self.prediction, 1))
        return tf.reduce_mean(tf.cast(mistakes, tf.float32))
    
    
    

    

In [15]:
def main(dataset, labels, params, mode = None):
    batch_size = params["batch_size"]
    sess = tf.Session()
    image = tf.placeholder(tf.float32, [None, params["img_size"], params["img_size"], params["num_channels"]])
    label = tf.placeholder(tf.float32, [None, params["num_classes"]])
    model = Model(image, label, params, mode = None)
    sess.run(tf.global_variables_initializer())
    num_steps = 1001
    with tf.Session() as session:
        tf.global_variables_initializer().run()
        print('Initialized')
        for step in range(num_steps):
            offset = (step*batch_size) % (labels.shape[0] - batch_size)
            batch_data = dataset[offset :(offset+ batch_size),:,:,:]
            batch_labels = labels[offset : (offset + batch_size),:]
            feed_dict = {image: batch_data, label: batch_labels}
            err = session.run(model.error, feed_dict= feed_dict)
            if (step % 100 == 0):
                print('Minibatch loss at step %d: %f' % (step, err))
                #print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
              
    


In [17]:
tf.reset_default_graph()
if __name__ == '__main__':
    main(x,y,params)

Initialized
Minibatch loss at step 0: 1.000000
Minibatch loss at step 100: 1.000000
Minibatch loss at step 200: 0.620000
Minibatch loss at step 300: 0.650000
Minibatch loss at step 400: 0.940000
Minibatch loss at step 500: 0.970000
Minibatch loss at step 600: 1.000000
Minibatch loss at step 700: 1.000000
Minibatch loss at step 800: 1.000000
Minibatch loss at step 900: 1.000000
Minibatch loss at step 1000: 0.820000
